# 🐘 WangChanGLM v0.1 demo

[![](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pythainlp/WangChanGLM/blob/main/demo/WangChanGLM_v0_1_demo.ipynb)

- Notebook นี้เขียนตัวอย่างการใช้งานโมเดล WangchanGPT ที่เป็นโมเดลภาษาขนาดใหญ่ (large language model) ที่ถูก fine-tune เพื่อให้ตอบคำถามผ่านการใส่ text prompt ได้
- สำหรับรายละเอียดการทำงานและการเทรนโมเดลสามารอ่านเพิ่มเติมได้จาก [Medium blog](https://medium.com/@iwishcognitivedissonance/7aa9a0f51f5f)
- Notebook พัฒนาภายใต้ license [Apache Software License 2.0](https://github.com/PyThaiNLP/pythainlp/blob/dev/LICENSE).
- Repository: [pythainlp/wangchanglm](https://github.com/PyThaiNLP/WangChanGLM)
- วิธีกาารรัน ให้ login บัญชี Google แล้ว
  - กดที่แถบ Runtime จากนั้นคลิก Run all
  - สำหรับภาษาไทยกดที่แถบ รันไทม์ จากนั้นคลิ้ก เรียกใช้ทุกเซลล์
  - คุณสามารถใช้ colab gpu ธรรมดาในการรัน notebook นี้ได้ ไม่จำเป็นต้องสมัคร colab pro

ผู้พัฒนา: [PyThaiNLP](https://github.com/PyThaiNLP) และ [VISTEC-depa AI Research Thailand](https://airesearch.in.th/)

![](https://avatars.githubusercontent.com/u/32934255?s=45&v=4)
![](https://airesearch.in.th/assets/img/logo/airesearch-logo.svg)

**ไม่รับประกันคำตอบที่ตอบกลับ**

In [ ]:
#@title Install
!pip install --q torch sentencepiece accelerate transformers pandas requests gradio bitsandbytes langchain faiss-gpu sentence-transformers gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.3/215.3 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 72.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 57.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.3/104.3 MB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 666.9/666.9 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 8.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 56.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.3/75.3 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 16.4 MB/s eta 0:0

In [ ]:
#@title Model
#@markdown WangChanGLM v0.1 มี 2 โมเดล คือ
#@markdown 
#@markdown 1. pythainlp/wangchanglm-7.5B-sft-en-sharded -> เป็นโมเดลที่เทรนด้วยภาษาอังกฤษอย่างเดียว (ดีที่สุด)
#@markdown 2. pythainlp/wangchanglm-7.5B-sft-enth-sharded -> โมเดลที่เทรนกับชุดข้อมูลที่เป็นภาษาไทย อาจจะดีกว่าโมเดลข้างบนในบางกรณี
#@markdown
#@markdown **หากคุณเปลี่ยนแปลงโมเดลที่รัน** หลังจากรันข้างล่างหมดแล้ว ให้คุณเลืแกโมเดลที่ต้องการและรันใหม่ตั้งแต่ต้นอีกครั้ง โดยให้กด Runtime -> Disconnect Colab Runtime แล้ว Run all หรือ รันไทม์ -> ยกเลิกการเชื่อมต่อและลบรันไทม์ แล้วคลิก เรียกใช้ทุกเซลล์
name_model = "pythainlp/wangchanglm-7.5B-sft-en-sharded" #@param ["pythainlp/wangchanglm-7.5B-sft-en-sharded","pythainlp/wangchanglm-7.5B-sft-enth-sharded"]

In [ ]:
#@title Download Model
import warnings
warnings.filterwarnings("ignore")
import gradio as gr
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from typing import Optional, Union, List, Dict, Any
import random
import time
import datetime
import os
import re
import pandas as pd

# name_model = "pythainlp/wangchanglm-7.5B-sft-adapter-merged-sharded"
model = AutoModelForCausalLM.from_pretrained(
    name_model, 
    return_dict=True, 
    load_in_8bit=True , # try use load_in_8bit=False for use fp16 :D
    device_map="auto", 
    torch_dtype=torch.float16, 
    offload_folder="./", 
    low_cpu_mem_usage=True,
)
tokenizer = AutoTokenizer.from_pretrained("facebook/xglm-7.5B")


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


Loading checkpoint shards:   0%|          | 0/98 [00:00<?, ?it/s]

## คู่มือการรันเบื้องต้น

### กำหนดภาษา

เนื่องจาก WangchanGPT เป็นโมเดล Multilingual Instruction-Following Model รองรับหลายภาษา ซึ่งอาจทำให้บางครั้งคำถามภาษาไทยอาจถูกตอบกลับเป็นภาษาอังกฤษได้ ดังนั้น เราจึงทำการแก้ไขปัญหาด้วยการบังคับโมเดลให้รันเฉฉพาะภาษาไทยเท่านั้น คุณสามารถปิดการตั้งค่านี้ได้ตามช่่อง "กำหนดภาษาสำหรับการตอบกลับ"

### หน้า ChatBot

![](https://i.imgur.com/rZiG1HA.png)

แท็บนี้เป็นหน้าต่างให้สนทนากับ AI ผ่านรูปแบบแชท ให้คุณพิมพ์ข้อความในช่องข้อความ "พิมพ์คำถามของคุณที่นี่... (กด enter หรือ submit หลังพิมพ์เสร็จ)" แล้วกด enter หรือ submit เพื่อให้ AI ตอบแชทของคุณ

สำหรับ Feedback คุณสามารถส่ง Feedback การตอบแชทของ AI ได้ โดยมี 3 ช่อง

1. Good - คำตอบดี ตรงกับคำถาม
2. Bad - แย่ คำตอบไม่ตรง ตอบผิด
3. Report - คำตอบออกมาไม่เหมาะสม หรือ ผิดพลาดอย่างร้ายแรง

และคุณสามารถเสนอแนะเพิ่มเติมในช่อง Feedback chatbot แล้วกด Submit Feedback เพื่อส่ง Feedback ของคุณ ถ้าสำเร็จจะขึ้น Thank you for feedback และช่องข้อความจะหายไป

### หน้า Text Generation

![](https://i.imgur.com/8uC2zbP.png)

เป็นหน้าสำหรับให้สร้างข้อความปรับแต่งค่าได้ โดยให้ใส่ค่าต่อไปนี้

- Instruction: สำหรับใส่คำสั่งที่ต้องการให้ AI ทำงาน
- Context input: สำหรับใส่เนื้อหาของคำสั่ง (ไม่จำเป็น)

จากนั้นยังสามารถกำหนดค่า Max new tokens ได้ และสามารถปรับแต่ง Advanced options ได้ ทั้งค่า Temperature และ Top p (ไม่จำเป็น)

หลังจากใส่ครบแล้ว สามารถกด Run ได้เลย ผลลัพธ์จะขึ้นที่ช่องทางขวามือ และมีช่อง Feedback

สำหรับ Feedback คุณสามารถส่ง Feedback การตอบแชทของ AI ได้ โดยมี 3 ช่อง

1. Good - คำตอบดี ตรงกับคำถาม
2. Bad - แย่ คำตอบไม่ตรง ตอบผิด
3. Report - คำตอบออกมาไม่เหมาะสม หรือ ผิดพลาดอย่างร้ายแรง

และคุณสามารถเสนอแนะเพิ่มเติมในช่อง Feedback chatbot แล้วกด Submit Feedback เพื่อส่ง Feedback ของคุณ ถ้าสำเร็จจะขึ้น Thank you for feedback และช่องข้อความจะหายไป

# Run the model

In [ ]:
#@title กำหนดภาษาสำหรับการตอบกลับ
#@markdown คุณต้องการให้ AI ตอบกลับเป็นภาษาไทยเท่านั้นหรือไม่? (Yes คือ ใช่ และ No คือ ไม่)
Thai = "Yes" #@param ["Yes", "No"]

## Run

In [ ]:
#@title Run
#@markdown วิธีการใช้งาน สำหรับแชทให้คลิกแท็บ chatbot ส่วนหากต้องการสร้างข้อความให้ไปที่ Text Generation
from langchain.llms import HuggingFacePipeline
from transformers import pipeline

from langchain import ConversationChain, LLMChain, PromptTemplate
from langchain.memory import ConversationBufferWindowMemory
import torch


from transformers import AutoTokenizer,AutoModelForCausalLM

template = """
{history}
<human>: {human_input}
<bot>:"""

prompt = PromptTemplate(
    input_variables=["history", "human_input"], 
    template=template
)
exclude_pattern = re.compile(r'[^ก-๙]+') #|[^0-9a-zA-Z]+
def is_exclude(text):
   return bool(exclude_pattern.search(text))

df = pd.DataFrame(tokenizer.vocab.items(), columns=['text', 'idx'])
df['is_exclude'] = df.text.map(is_exclude)
exclude_ids = df[df.is_exclude==True].idx.tolist()
if Thai=="Yes":
  pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    begin_suppress_tokens=exclude_ids,
    no_repeat_ngram_size=2,
  )
else:
  pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    no_repeat_ngram_size=2,
  )
hf_pipeline = HuggingFacePipeline(pipeline=pipe)

chatgpt_chain = LLMChain(
    llm=hf_pipeline, 
    prompt=prompt, 
    verbose=True, 
    memory=ConversationBufferWindowMemory(k=2),
)


api_url = "https://wangchanglm.numfa.com/api.php" # Don't open this url!!!
import requests
from urllib.request import urlopen
from urllib.parse import urlencode
from urllib.error import HTTPError, URLError
from urllib.request import Request
import copy

def sumbit_data(save,prompt,vote,feedback=None,max_len=None,temp=None,top_p=None,name_model=name_model):
  api_url = "https://wangchanglm.numfa.com/api.php" 
  myobj = {
      'save': save,
      'prompt':prompt,
      'vote':vote,
      'feedback':feedback,
      'max_len':max_len,
      'temp':temp,
      'top_p':top_p,
      'model':name_model
  }
  _temp_url ="https://wangchanglm.numfa.com/api.php" 
  _temp_url += "?" + urlencode(myobj, doseq=True, safe="/")
  html = urlopen(_temp_url).read().decode('utf-8')
  return True


def gen_instruct(text,max_new_tokens=512,top_p=0.95,temperature=0.9,top_k=50):
    batch = tokenizer(text, return_tensors="pt")
    with torch.cuda.amp.autocast(): # cuda -> cpu if cpu
        if Thai=="Yes":
          output_tokens = model.generate(
            input_ids=batch["input_ids"],
            max_new_tokens=max_new_tokens, # 512
            begin_suppress_tokens = exclude_ids,
            no_repeat_ngram_size=2,
            #oasst k50
            top_k=top_k,
            top_p=top_p, # 0.95
            typical_p=1.,
            temperature=temperature, # 0.9
          )
        else:
          output_tokens = model.generate(
            input_ids=batch["input_ids"],
            max_new_tokens=max_new_tokens, # 512
            no_repeat_ngram_size=2,
            #oasst k50
            top_k=top_k,
            top_p=top_p, # 0.95
            typical_p=1.,
            temperature=temperature, # 0.9
          )
    return tokenizer.decode(output_tokens[0][len(batch["input_ids"][0]):], skip_special_tokens=True)

def gen_chatbot_old(text):
    batch = tokenizer(text, return_tensors="pt")
    #context_tokens = tokenizer(text, add_special_tokens=False)['input_ids']
    #logits_processor = FocusContextProcessor(context_tokens, model.config.vocab_size, scaling_factor = 1.5)
    with torch.cpu.amp.autocast(): # cuda if gpu
        output_tokens = model.generate(
            input_ids=batch["input_ids"],
            max_new_tokens=512,
            begin_suppress_tokens = exclude_ids,
            no_repeat_ngram_size=2,
        )
    return tokenizer.decode(output_tokens[0], skip_special_tokens=True).split(": ")[-1]

def list2prompt(history):
    _text = ""
    for user,bot in history:
        _text+="<human>: "+user+"\n<bot>: "
        if bot!=None:
            _text+=bot+"\n"
    return _text

PROMPT_DICT = {
    "prompt_input": (
        "<context>: {input}\n<human>: {instruction}\n<bot>: "
    ),
    "prompt_no_input": (
        "<human>: {instruction}\n<bot>: "
    ),
}

def instruct_generate(
    instruct: str,
    input: str = 'none',
    max_gen_len=512,
    temperature: float = 0.1,
    top_p: float = 0.75,
):
    if input == 'none' or len(input)<2:
        prompt = PROMPT_DICT['prompt_no_input'].format_map(
            {'instruction': instruct, 'input': ''})
    else:
        prompt = PROMPT_DICT['prompt_input'].format_map(
            {'instruction': instruct, 'input': input})
    result = gen_instruct(prompt,max_gen_len,top_p,temperature)
    return result

with gr.Blocks(height=900) as demo:
    chatgpt_chain = LLMChain(
        llm=hf_pipeline, 
        prompt=prompt, 
        verbose=True, 
        memory=ConversationBufferWindowMemory(k=2),
    )
    with gr.Tab("Text Generation"):
        with gr.Row():
            with gr.Column():
                instruction = gr.Textbox(lines=2, label="Instruction",max_lines=10)
                input = gr.Textbox(
                    lines=2, label="Context input", placeholder='none',max_lines=5)
                max_len = gr.Slider(minimum=1, maximum=1024,
                                    value=512, label="Max new tokens")
                with gr.Accordion(label='Advanced options', open=False):
                    temp = gr.Slider(minimum=0, maximum=1,
                                     value=0.9, label="Temperature")
                    top_p = gr.Slider(minimum=0, maximum=1,
                                      value=0.95, label="Top p")

                run_botton = gr.Button("Run")

            with gr.Column():
                outputs = gr.Textbox(lines=10, label="Output")
                with gr.Column(visible=False) as feedback_gen_box:
                    gen_radio = gr.Radio(
                        ["Good", "Bad", "Report"], label="Do you think about the chat?")
                    feedback_gen = gr.Textbox(placeholder="Feedback chatbot",show_label=False, lines=4)
                    feedback_gen_submit = gr.Button("Submit Feedback")
                with gr.Row(visible=False) as feedback_gen_ok:
                    gr.Markdown("Thank you for feedback.")

        def save_up2(instruction, input,prompt,max_len,temp,top_p,choice,feedback):
            save="gen"
            if input == 'none' or len(input)<2:
              _prompt = PROMPT_DICT['prompt_no_input'].format_map(
                  {'instruction': instruction, 'input': ''})
            else:
              _prompt = PROMPT_DICT['prompt_input'].format_map(
                  {'instruction': instruction, 'input': input})
            prompt=_prompt+prompt
            if choice=="Good":
              sumbit_data(save=save,prompt=prompt,vote=1,feedback=feedback,max_len=max_len,temp=temp,top_p=top_p)
            elif choice=="Bad":
              sumbit_data(save=save,prompt=prompt,vote=0,feedback=feedback,max_len=max_len,temp=temp,top_p=top_p)
            else:
              sumbit_data(save=save,prompt=prompt,vote=3,feedback=feedback,max_len=max_len,temp=temp,top_p=top_p)
            return {feedback_gen_box: gr.update(visible=False),feedback_gen_ok: gr.update(visible=True)}
        def gen(instruct: str,input: str = 'none',max_gen_len=512,temperature: float = 0.1,top_p: float = 0.75):
            feedback_gen_ok.update(visible=False)
            _temp= instruct_generate(instruct,input,max_gen_len,temperature,top_p)
            feedback_gen_box.update(visible=True)
            return {outputs:_temp,feedback_gen_box: gr.update(visible=True),feedback_gen_ok: gr.update(visible=False)}
        feedback_gen_submit.click(fn=save_up2, inputs=[instruction, input,outputs,max_len,temp,top_p,gen_radio,feedback_gen], outputs=[feedback_gen_box,feedback_gen_ok], queue=True)
        inputs = [instruction, input, max_len, temp, top_p]
        run_botton.click(fn=gen, inputs=inputs, outputs=[outputs,feedback_gen_box,feedback_gen_ok])
        examples = gr.Examples(examples=[("แต่งกลอนวันแม่"), ("แต่งกลอนแปดวันแม่")],inputs=[instruction])
    with gr.Tab("ChatBot"):
        with gr.Column():
            chatbot = gr.Chatbot(label="Chat Message Box", placeholder="Chat Message Box",show_label=False).style(container=False)
        with gr.Row():
          with gr.Column(scale=0.85):
            msg = gr.Textbox(placeholder="พิมพ์คำถามของคุณที่นี่... (กด enter หรือ submit หลังพิมพ์เสร็จ)",show_label=False)
          with gr.Column(scale=0.15, min_width=0):
            submit = gr.Button("Submit")
        with gr.Column():
            with gr.Column(visible=False) as feedback_chatbot_box:
                chatbot_radio = gr.Radio(
                    ["Good", "Bad", "Report"], label="Do you think about the chat?"
                )
                feedback_chatbot = gr.Textbox(placeholder="Feedback chatbot",show_label=False, lines=4)
                feedback_chatbot_submit = gr.Button("Submit Feedback")
            with gr.Row(visible=False) as feedback_chatbot_ok:
                gr.Markdown("Thank you for feedback.")
        clear = gr.Button("Clear")
        def save_up(history,choice,feedback):
            _bot = list2prompt(history)
            x=False
            if choice=="Good":
              x=sumbit_data(save="chat",prompt=_bot,vote=1,feedback=feedback)
            elif choice=="Bad":
              x=sumbit_data(save="chat",prompt=_bot,vote=0,feedback=feedback)
            else:
              x=sumbit_data(save="chat",prompt=_bot,vote=3,feedback=feedback)
            return {feedback_chatbot_ok: gr.update(visible=True),feedback_chatbot_box: gr.update(visible=False)}
        def user(user_message, history):
            bot_message = chatgpt_chain.predict(human_input=user_message)
            history.append((user_message, bot_message))
            return "", history,gr.update(visible=True)
        def reset():
          chatgpt_chain.memory.clear()
          print("clear!")
        feedback_chatbot_submit.click(fn=save_up, inputs=[chatbot,chatbot_radio,feedback_chatbot], outputs=[feedback_chatbot_ok,feedback_chatbot_box,], queue=True)
        clear.click(reset, None, chatbot, queue=False)
        submit_event = msg.submit(fn=user, inputs=[msg, chatbot], outputs=[msg, chatbot,feedback_chatbot_box], queue=True)
        submit_click_event = submit.click(fn=user, inputs=[msg, chatbot], outputs=[msg, chatbot,feedback_chatbot_box], queue=True)
    with gr.Tab("ChatBot without LangChain"):
        chatbot2 = gr.Chatbot()
        msg2 = gr.Textbox(label="Your sentence here... (press enter to submit)")
        with gr.Column():
            with gr.Column(visible=False) as feedback_chatbot_box2:
                chatbot_radio2 = gr.Radio(
                    ["Good", "Bad", "Report"], label="Do you think about the chat?"
                )
                feedback_chatbot2 = gr.Textbox(placeholder="Feedback chatbot",show_label=False, lines=4)
                feedback_chatbot_submit2 = gr.Button("Submit Feedback")
            with gr.Row(visible=False) as feedback_chatbot_ok2:
                gr.Markdown("Thank you for feedback.")
        
        def user2(user_message, history):
            return "", history + [[user_message, None]]
        def bot2(history):
            _bot = list2prompt(history)
            bot_message = gen_chatbot_old(_bot)
            history[-1][1] = bot_message
            return history,gr.update(visible=True)
        def save_up2(history,choice,feedback):
            _bot = list2prompt(history)
            x=False
            if choice=="Good":
              x=sumbit_data(save="chat",prompt=_bot,vote=1,feedback=feedback,name_model=name_model+"-chat_old")
            elif choice=="Bad":
              x=sumbit_data(save="chat",prompt=_bot,vote=0,feedback=feedback,name_model=name_model+"-chat_old")
            else:
              x=sumbit_data(save="chat",prompt=_bot,vote=3,feedback=feedback,name_model=name_model+"-chat_old")
            return {feedback_chatbot_ok2: gr.update(visible=True),feedback_chatbot_box2: gr.update(visible=False)}
        msg2.submit(user2, [msg2, chatbot2], [msg2, chatbot2], queue=False).then(bot2, chatbot2, [chatbot2,feedback_chatbot_box2])
        feedback_chatbot_submit2.click(fn=save_up2, inputs=[chatbot2,chatbot_radio2,feedback_chatbot2], outputs=[feedback_chatbot_ok2,feedback_chatbot_box2], queue=True)
        clear2 = gr.Button("Clear")
        clear2.click(lambda: None, None, chatbot2, queue=False)
demo.queue()
demo.launch(debug=True)

ตัวอย่างเพิ่มเติม

- [🦜⛓️ LangChain + 🐘 WangchanGLM](https://colab.research.google.com/drive/1pEZdM8lUVBIvgInbG2K7O-IdCX5fLwy3#scrollTo=GG_A9G4kHUBM)

ขอความร่วมมือท่านช่วยส่ง feedback การใช้งานกลับมา เพื่อให้เราพัฒนาโมเดลรุ่นหน้าออกมาที่ดีกว่าเดิมให้ใช้งานฟรี

คุณสามารถติดตามข่าวสารเกี่ยวกับ 🐘 WangChanGLM ได้ผ่านทาง Facebook: [AIResearch.in.th](https://www.facebook.com/AIResearch.in.th) กับ [PyThaiNLP](https://www.facebook.com/pythainlp)

**ข่าว**

เตรียมตัวมาพบกันพฤษภาคมนี้!

VISTEC AI Day 2023
วันที่ 22 พฤษภาคม 13:30-16:30 น.
The Synergy Hall, ชั้น 6, Energy Complex, Building C
พบกับเทคโนโลยีปัญญาประดิษฐ์ชั้นแนวหน้าจาก VISTEC ครั้งใหญ่
ที่จะแสดงศักยภาพจากงานวิจัยเชิงลึกสู่ประโยชน์เชิงธุรกิจ
พบกับบริษัท หน่วยวิจัย และงานวิจัยทางด้านปัญญาประดิษฐ์สุดล้ำ:
- VISAI.aiผู้ให้บริการ AI ครบวงจรสำหรับภาคธุรกิจ
Wangchan Advance Industrial Labs: แล็บวิจัยพัฒนาเทคโนโลยี ปัญญาประดิษฐ์ หุ่นยนต์ และเซนเซอร์
- WangchanX แนะนำโปรแกรมวิจัยพัฒนาโมเดลภาษาขนาดใหญ่ ที่มียอดดาวน์โหลดกว่า 1.2 ล้านครั้ง และการเปิดตัว WangChanGLM โมเดลภาษาเชิง generative ที่รองรับภาษาไทย และภาษาอื่น ๆ หลากหลายภาษา
- VISTEC-Siriraj Frontier Research Center ศูนย์วิจัยชั้นแนวหน้าสำหรับเทคโนโลยีการแพทย์ในอนาคต 
- SensAI เทคโนโลยีเซนเซอร์อัจฉริยะสำหรับการแพทย์เชิงป้องกันและการดูแลผู้ป่วยทางไกล
มาร่วมพบปะกับผู้เชี่ยวชาญ AI และบุคลากรสายเทค พร้อมเยี่ยมชมบูธได้ฟรี
ลงทะเบียนเข้าร่วมงานได้ที่ https://forms.office.com/r/y2nbppaG9g 

ติดต่อสอบถามเพิ่มเติมได้ที่ wachirawish_pro@vistec.ac.th
